In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/affairsdata/affairs.csv


In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 15.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4547baf2ce907a1753e986056e5de78acd0c0c9ab028466fe44a8a8e55806411
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
#import spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('random_forest').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/19 11:37:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#import libs
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel

In [5]:
#read the dataset
df = spark.read.csv('/kaggle/input/affairsdata/affairs.csv', header=True, inferSchema=True)
df.show(5)

+-------------+----+-----------+--------+---------+----+----------+---------------+------+
|rate_marriage| age|yrs_married|children|religious|educ|occupation|occupation_husb|affair|
+-------------+----+-----------+--------+---------+----+----------+---------------+------+
|            3|32.0|        9.0|     3.0|        3|  17|         2|              5|     1|
|            3|27.0|       13.0|     3.0|        1|  14|         3|              4|     1|
|            4|22.0|        2.5|     0.0|        1|  16|         3|              5|     1|
|            4|37.0|       16.5|     4.0|        3|  16|         5|              5|     1|
|            5|27.0|        9.0|     1.0|        1|  14|         3|              4|     1|
+-------------+----+-----------+--------+---------+----+----------+---------------+------+
only showing top 5 rows



In [6]:
df = df.drop('educ','occupation','occupation_husb')
df.show(5,False)

+-------------+----+-----------+--------+---------+------+
|rate_marriage|age |yrs_married|children|religious|affair|
+-------------+----+-----------+--------+---------+------+
|3            |32.0|9.0        |3.0     |3        |1     |
|3            |27.0|13.0       |3.0     |1        |1     |
|4            |22.0|2.5        |0.0     |1        |1     |
|4            |37.0|16.5       |4.0     |3        |1     |
|5            |27.0|9.0        |1.0     |1        |1     |
+-------------+----+-----------+--------+---------+------+
only showing top 5 rows



In [7]:
print(((df.count()), (len(df.columns))))

(6366, 6)


In [8]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affair: integer (nullable = true)



### EDA

In [9]:
df.describe().select('summary', 'rate_marriage', 'age', 'yrs_married', 'children', 'religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655023| 6.847881883668814|7.280119972766427| 1.433470828560344|0.8783688402641788|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [10]:
df.groupBy('affair').count().show()

+------+-----+
|affair|count|
+------+-----+
|     1| 2053|
|     0| 4313|
+------+-----+



In [11]:
df.groupBy('rate_marriage').count().show()

+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            1|   99|
|            3|  993|
|            5| 2684|
|            4| 2242|
|            2|  348|
+-------------+-----+



In [12]:
df.groupby('rate_marriage', 'affair').count().orderBy('rate_marriage', 'affair', 'count').show()

+-------------+------+-----+
|rate_marriage|affair|count|
+-------------+------+-----+
|            1|     0|   25|
|            1|     1|   74|
|            2|     0|  127|
|            2|     1|  221|
|            3|     0|  446|
|            3|     1|  547|
|            4|     0| 1518|
|            4|     1|  724|
|            5|     0| 2197|
|            5|     1|  487|
+-------------+------+-----+



In [13]:
df.groupby('religious', 'affair').count().orderBy('religious', 'affair', 'count').show()

+---------+------+-----+
|religious|affair|count|
+---------+------+-----+
|        1|     0|  613|
|        1|     1|  408|
|        2|     0| 1448|
|        2|     1|  819|
|        3|     0| 1715|
|        3|     1|  707|
|        4|     0|  537|
|        4|     1|  119|
+---------+------+-----+



In [14]:
df.groupby('children', 'affair').count().orderBy('children', 'affair', 'count').show()

+--------+------+-----+
|children|affair|count|
+--------+------+-----+
|     0.0|     0| 1912|
|     0.0|     1|  502|
|     1.0|     0|  747|
|     1.0|     1|  412|
|     2.0|     0|  873|
|     2.0|     1|  608|
|     3.0|     0|  460|
|     3.0|     1|  321|
|     4.0|     0|  197|
|     4.0|     1|  131|
|     5.5|     0|  124|
|     5.5|     1|   79|
+--------+------+-----+



In [15]:
df.groupby('affair').mean().show()

+------+------------------+------------------+------------------+------------------+------------------+-----------+
|affair|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affair)|
+------+------------------+------------------+------------------+------------------+------------------+-----------+
|     1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|        1.0|
|     0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|        0.0|
+------+------------------+------------------+------------------+------------------+------------------+-----------+



In [16]:
df.columns

['rate_marriage', 'age', 'yrs_married', 'children', 'religious', 'affair']

In [17]:
df_assemble = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], 
                              outputCol='features')
df = df_assemble.transform(df)
df.show(5,False)

+-------------+----+-----------+--------+---------+------+-----------------------+
|rate_marriage|age |yrs_married|children|religious|affair|features               |
+-------------+----+-----------+--------+---------+------+-----------------------+
|3            |32.0|9.0        |3.0     |3        |1     |[3.0,32.0,9.0,3.0,3.0] |
|3            |27.0|13.0       |3.0     |1        |1     |[3.0,27.0,13.0,3.0,1.0]|
|4            |22.0|2.5        |0.0     |1        |1     |[4.0,22.0,2.5,0.0,1.0] |
|4            |37.0|16.5       |4.0     |3        |1     |[4.0,37.0,16.5,4.0,3.0]|
|5            |27.0|9.0        |1.0     |1        |1     |[5.0,27.0,9.0,1.0,1.0] |
+-------------+----+-----------+--------+---------+------+-----------------------+
only showing top 5 rows



In [18]:
#select data for model building
model_df = df.select(['features', 'affair'])
model_df.show(5,False)

+-----------------------+------+
|features               |affair|
+-----------------------+------+
|[3.0,32.0,9.0,3.0,3.0] |1     |
|[3.0,27.0,13.0,3.0,1.0]|1     |
|[4.0,22.0,2.5,0.0,1.0] |1     |
|[4.0,37.0,16.5,4.0,3.0]|1     |
|[5.0,27.0,9.0,1.0,1.0] |1     |
+-----------------------+------+
only showing top 5 rows



In [19]:
#train, test split
traindf, testdf = model_df.randomSplit([0.75, 0.25], seed=2)

In [20]:
traindf.count()

4801

In [21]:
traindf.groupby('affair').count().show()

+------+-----+
|affair|count|
+------+-----+
|     1| 1554|
|     0| 3247|
+------+-----+



In [22]:
testdf.count()

1565

In [23]:
testdf.groupby('affair').count().show()

+------+-----+
|affair|count|
+------+-----+
|     1|  499|
|     0| 1066|
+------+-----+



In [24]:
rf_clasifier = RandomForestClassifier(labelCol='affair', numTrees=50).fit(traindf)

In [25]:
rf_pred = rf_clasifier.transform(testdf)
rf_pred.show(5, False)

+-----------------------+------+---------------------------------------+----------------------------------------+----------+
|features               |affair|rawPrediction                          |probability                             |prediction|
+-----------------------+------+---------------------------------------+----------------------------------------+----------+
|[1.0,22.0,2.5,1.0,1.0] |1     |[23.049534773954225,26.950465226045775]|[0.4609906954790845,0.5390093045209156] |1.0       |
|[1.0,27.0,2.5,0.0,2.0] |1     |[22.550380188783063,27.449619811216937]|[0.45100760377566124,0.5489923962243387]|1.0       |
|[1.0,27.0,6.0,0.0,2.0] |0     |[20.20978052154694,29.790219478453064] |[0.40419561043093877,0.5958043895690612]|1.0       |
|[1.0,27.0,9.0,1.0,3.0] |1     |[16.59447666316973,33.40552333683027]  |[0.3318895332633946,0.6681104667366055] |1.0       |
|[1.0,27.0,13.0,2.0,2.0]|1     |[13.391973181524843,36.60802681847516] |[0.2678394636304969,0.7321605363695032] |1.0       |


In [26]:
rf_pred.groupby('Prediction').count().show()

+----------+-----+
|Prediction|count|
+----------+-----+
|       0.0| 1282|
|       1.0|  283|
+----------+-----+



In [27]:
rf_accuracy = MulticlassClassificationEvaluator(labelCol='affair', metricName='accuracy').evaluate(rf_pred)
rf_precision = MulticlassClassificationEvaluator(labelCol='affair', metricName='weightedPrecision').evaluate(rf_pred)
rf_f1 = MulticlassClassificationEvaluator(labelCol='affair', metricName='f1').evaluate(rf_pred)
rf_auc=BinaryClassificationEvaluator(labelCol='affair').evaluate(rf_pred)

In [28]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 74%


In [29]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 73%


In [30]:
print('The f1 rate on test data is {0:.0%}'.format(rf_f1))

The f1 rate on test data is 72%


In [31]:
print(rf_auc)

0.7687344670579435


In [32]:
#feature importance
rf_clasifier.featureImportances

SparseVector(5, {0: 0.6176, 1: 0.0307, 2: 0.2346, 3: 0.0501, 4: 0.0669})

In [33]:
df.schema['features'].metadata['ml_attr']['attrs']

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

In [34]:
#save the model
rf_clasifier.save('./RF_model')

In [35]:
rf = RandomForestClassificationModel.load('./RF_model')

In [36]:
model_pred = rf.transform(testdf)
model_pred.show(5,False)

+-----------------------+------+---------------------------------------+----------------------------------------+----------+
|features               |affair|rawPrediction                          |probability                             |prediction|
+-----------------------+------+---------------------------------------+----------------------------------------+----------+
|[1.0,22.0,2.5,1.0,1.0] |1     |[23.049534773954225,26.950465226045775]|[0.4609906954790845,0.5390093045209156] |1.0       |
|[1.0,27.0,2.5,0.0,2.0] |1     |[22.550380188783063,27.449619811216937]|[0.45100760377566124,0.5489923962243387]|1.0       |
|[1.0,27.0,6.0,0.0,2.0] |0     |[20.20978052154694,29.790219478453064] |[0.40419561043093877,0.5958043895690612]|1.0       |
|[1.0,27.0,9.0,1.0,3.0] |1     |[16.59447666316973,33.40552333683027]  |[0.3318895332633946,0.6681104667366055] |1.0       |
|[1.0,27.0,13.0,2.0,2.0]|1     |[13.391973181524843,36.60802681847516] |[0.2678394636304969,0.7321605363695032] |1.0       |
